In [1]:
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a%b)

def whole_vector(a, b):
    ax, ay = a
    bx, by = b
    vx = abs(bx - ax)
    vy = abs(by - ay)
    div = max(1, gcd(max(vx, vy), min(vx, vy)))
    vx = bx - ax
    vy = by - ay
    return (int(vx/div), int(vy/div))

def coord_dict(img):
    d = dict()
    y = 0
    for line in img:
        x = 0
        for c in line.strip():
            if c == '#':
                d[(x,y)] = 0
            x += 1
        y += 1
    return d

def los(img):
    d = coord_dict(img)
    for a in d:
        c = set()
        for b in d:
            c.add(whole_vector(a, b))
        d[a] = len(c) - 1
    best = a
    for k, v in d.items():
        if v > d[best]:
            best = k
    return best, d[best]

def clockwise(a, b):
    ax, ay = a
    bx, by = b
    return ay * bx <= ax * by

def sort_lists_on_dist_from_best(lists, best):
    def dist(b):
        ax, ay = best
        bx, by = b
        return ((bx - ax) ** 2 + (by - ay) ** 2) ** 0.5
    
    for key in lists:
        if len(lists[key]) > 1:
            lists[key] = sorted(lists[key], key=dist)
    return lists

def rel_dict(img, best):
    yadda = dict()
    y = 0
    for line in img:
        x = 0
        for c in line.strip():
            if c == '#':
                vx, vy = whole_vector(best, (x, y))
                if (vx, vy) not in yadda:
                    yadda[(vx, vy)] = []
                yadda[(vx, vy)].append((x,y))
            x += 1
        y += 1
    return yadda

def ticktocksort(coords):
    if len(coords) <= 1:
        return coords
    left = []
    right = []
    center = coords[0]
    for coord in coords[1:]:
        if clockwise(center, coord):
            right.append(coord)
        else:
            left.append(coord)
    return ticktocksort(left) + [center] + ticktocksort(right)

def order_coords(lists):
    left = []
    right = []
    for (x, y) in lists:
        if x < 0:
            left.append((x,y))
        elif x > 0:
            right.append((x,y))
        elif y < 0:
            right.append((x,y))
        elif y > 0:
            left.append((x,y))
        #no 'else': conveniently skip over itself
    return ticktocksort(right) + ticktocksort(left)

def asteroid_order(img):
    best = los(img)[0]
    lists = sort_lists_on_dist_from_best(rel_dict(img, best), best)
    order = []
    while lists:
        order += [lists[o][0] for o in order_coords(lists)]
        lists = {i:lists[i][1:] for i in lists if len(lists[i]) > 1}
    return order

In [2]:
map1 = ['.#..#',
        '.....',
        '#####',
        '....#',
        '...##']
los(map1)

((3, 4), 8)

In [3]:
map2 = ['......#.#.',
        '#..#.#....',
        '..#######.',
        '.#.#.###..',
        '.#..#.....',
        '..#....#.#',
        '#..#....#.',
        '.##.#..###',
        '##...#..#.',
        '.#....####']
los(map2)

((5, 8), 33)

In [4]:
map3 = ['#.#...#.#.',
        '.###....#.',
        '.#....#...',
        '##.#.#.#.#',
        '....#.#.#.',
        '.##..###.#',
        '..#...##..',
        '..##....##',
        '......#...',
        '.####.###.']
los(map3)

((1, 2), 35)

In [5]:
map4 = ['.#..#..###',
        '####.###.#',
        '....###.#.',
        '..###.##.#',
        '##.##.#.#.',
        '....###..#',
        '..#.#..#.#',
        '#..#.#.###',
        '.##...##.#',
        '.....#.#..']
los(map4)

((6, 3), 41)

In [6]:
map5 = ['.#..##.###...#######',
        '##.############..##.',
        '.#.######.########.#',
        '.###.#######.####.#.',
        '#####.##.#.##.###.##',
        '..#####..#.#########',
        '####################',
        '#.####....###.#.#.##',
        '##.#################',
        '#####.##.###..####..',
        '..######..##.#######',
        '####.##.####...##..#',
        '.#####..#.######.###',
        '##...#.##########...',
        '#.##########.#######',
        '.####.#.###.###.#.##',
        '....##.##.###..#####',
        '.#.#.###########.###',
        '#.#.#.#####.####.###',
        '###.##.####.##.#..##']
los(map5)

((11, 13), 210)

In [7]:
with open("input10.txt") as infile:
    img = infile.readlines()
los(img)

((29, 28), 256)

In [8]:
with open("input10.txt") as infile:
    img = infile.readlines()
asteroid_order(img)[199]

(17, 7)